In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32040
15
['amit', 'eric', 'kevin', 'moxa', 'parneet', 'nico', 'xinyan', 'emma']


In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
block_size = 3
# construct the dataset
def build_dataset(words, block_size = 3, do_print=False):
    X, Y = [], []
    for word in words:
        context = [0] * block_size
        for w in word + '.':
            X.append(context)
            Y.append(stoi[w])
            context = context[1:] + [stoi[w]]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    if do_print:
        print(X.shape)
        print(Y.shape)
        for row in range(X.shape[0]):
            xes = [itos[X[row][i].item()] for i in range(X.shape[1])]
            y = itos[Y[row].item()]
            print(f"prediction for {''.join(xes)}: {y}")
    return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte, Yte = build_dataset(words[n2:])


In [5]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embd = 10 # dimensionality of character embedding vectors
n_hidden = 64 # number of neurons in MLP hidden layer

g = torch.Generator().manual_seed(1337)
C = torch.randn((vocab_size, n_embd))
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # num params in total
for p in parameters:
    p.requires_grad = True



4137


In [7]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [8]:
W2.shape

torch.Size([64, 27])

In [9]:
# print(C.shape)
# print(Xb.shape)
# print(emb.view(32, -1).shape)
# print(Xb[0])
# print(C[Xb[0][0]])
# print(C[0])
Yb

tensor([ 9,  9, 10,  5, 25,  5,  5, 12, 25,  1, 12,  1, 18, 14,  9,  1, 14,  9,
        19, 15,  8,  0,  1,  0,  9,  1, 10,  0, 10,  0,  7,  9])

In [10]:
# expanded out forward pass

emb = C[Xb] # embed chars into vectors
embcat = emb.view(emb.shape[0], -1) # concat the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# Batchnorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # bessel's correction
bnvar_inv=(bnvar+1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# nonlinearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross-entropy loss (same as F.cross_entropy(logits, Yb))
logits_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logits_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
    norm_logits, logits_maxes, logits, h, hpreact, bnraw,
    bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
    embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3134, grad_fn=<NegBackward0>)

In [11]:
# scratchpad
# logprobs[range(n)]

dlogprobs = torch.zeros(logprobs.shape)
dlogprobs[range(n), Yb] = -1/n


In [12]:
logits.shape

torch.Size([32, 27])

In [13]:
dprobs = 1 / probs * dlogprobs
dcounts_sum_inv = (dprobs * counts).sum(dim=1, keepdim=True)
dcounts_sum = -counts_sum ** -2 * dcounts_sum_inv
dcounts = dprobs * counts_sum_inv.expand_as(counts) + dcounts_sum
dnorm_logits = counts * dcounts
dlogit_maxes = -dnorm_logits.sum(dim=1, keepdim=True)
dlogits = (logits == logits_maxes.expand_as(logits)).float() * dlogit_maxes + dnorm_logits

In [20]:
# dh = W2.sum(1, keepdim=True).transpose(1, 0).expand_as(h) 
print(logits.shape)
print(h.shape)
print(W2.shape)

torch.Size([32, 27])
torch.Size([32, 64])
torch.Size([64, 27])


In [15]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logits_maxes)
cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0


In [16]:
# logprobs.shape
range(n)

range(0, 32)